## Download Datasets

### Download Cityscapes

In [1]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/cityscapes-correctlabels")

print("Path to dataset files:", path)

Mounting files to /kaggle/input/cityscapes-correctlabels...
Path to dataset files: /kaggle/input/cityscapes-correctlabels


### Download Tiny Imagenet

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/tiny-imagenet


### Download fishyscapes- lost and found

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/fs-l-and-f")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fs-l-and-f


### Download SMIYC Road Anomaly 21

In [2]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/smiyc-road-anomaly-21")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/smiyc-road-anomaly-21


### Download SMIYC Road Obstacles 21

In [1]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/smiyc-road-obstacles-21")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/smiyc-road-obstacles-21


## Git Clone

In [1]:
!git clone -b prova-2 https://github.com/Nome-Provvisorio/AnomalySegmentation.git

Cloning into 'AnomalySegmentation'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 245 (delta 38), reused 0 (delta 0), pack-reused 189 (from 1)
Receiving objects: 100% (245/245), 79.75 MiB | 40.95 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [3]:
!git pull origin main

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [5]:
!pip install visdom
!pip install cv2
#!pip install torch_xla

#!pip install cityscapesscripts
!pip install ood_metrics


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Run Pretraining

In [1]:
%cd "/kaggle/working"
!python3 "AnomalySegmentation/imagenet/main.py" "/kaggle/input/tiny-imagenet/tiny-imagenet-200"

/kaggle/working
python3: can't open file '/kaggle/working/AnomalySegmentation/imagenet/main.py': [Errno 2] No such file or directory


## Run training

In [3]:
%cd "/kaggle/working/AnomalySegmentation/train"
!python3 "main.py" --savedir erfnet_training1 --datadir "/kaggle/input/cityscapes-correctlabels/Cityscape" --num-epochs 10 --batch-size 1 --decoder --pretrainedEncoder "/kaggle/working/AnomalySegmentation/train/model_best.pth.tar"

/kaggle/working/AnomalySegmentation/train
model path is:  erfnet
Current working directory: /kaggle/working/AnomalySegmentation/train
========== DECODER TRAINING ===========
Loading encoder pretrained in imagenet
/kaggle/working/AnomalySegmentation/train/main.py:492: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case

## Convertitore Labels

In [ ]:

# python imports
from __future__ import print_function, absolute_import, division
import os, glob, sys

# cityscapes imports
from cityscapesscripts.helpers.csHelpers import printError
from cityscapesscripts.preparation.json2labelImg import json2labelImg

# The main method
def main():
    # Where to look for Cityscapes

    cityscapesPath = "/kaggle/input/cityscapes/Cityscape"
    # how to search for all ground truth
    searchFine   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_polygons.json" )
    searchCoarse = os.path.join( cityscapesPath , "gtCoarse" , "*" , "*" , "*_gt*_polygons.json" )

    # search files
    filesFine = glob.glob( searchFine )
    filesFine.sort()
    filesCoarse = glob.glob( searchCoarse )
    filesCoarse.sort()

    # concatenate fine and coarse
    files = filesFine + filesCoarse
    # files = filesFine # use this line if fine is enough for now.

    # quit if we did not find anything
    if not files:
        printError( "Did not find any files. Please consult the README." )

    # a bit verbose
    print("Processing {} annotation files".format(len(files)))

    # iterate through files
    progress = 0
    print("Progress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
    for f in files:
        # create the output filename
        dst = f.replace( "_polygons.json" , "_labelTrainIds.png" )

        # do the conversion
        try:
            json2labelImg( f , dst , "trainIds" )
        except:
            print("Failed to convert: {}".format(f))
            raise

        # status
        progress += 1
        print("\rProgress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
        sys.stdout.flush()


# call the main
if __name__ == "__main__":
    main()

## Evaluation

### Generates color images for visualization

In [ ]:
!python3 "/kaggle/working/AnomalySegmentation/eval/eval_cityscapes_color.py" --datadir "/kaggle/input/cityscapes-correctlabels/Cityscape" --subset val --loadDir "/kaggle/working/" --loadWeights "AnomalySegmentation/save/erfnet_training1/model_best.pth" 

### eval_iou

In [ ]:
!python3  "/kaggle/working/AnomalySegmentation/eval/eval_iou.py" --datadir "/kaggle/input/cityscapes-correctlabels/Cityscape" --subset val --loadDir "/kaggle/working/" --loadWeights "AnomalySegmentation/save/erfnet_training1/model_best.pth"

In [7]:
!pip install opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.7 MB/s eta 0:00:24
   ---------------------------------------- 0.5/38.8 MB 5.9 MB/s eta 0:00:07
   - -------------------------------------- 1.8/38.8 MB 14.3 MB/s eta 0:00:03
   --- ------------------------------------ 3.1/38.8 MB 18.2 MB/s eta 0:00:02
   ----- ---------------------------------- 5.6/38.8 MB 25.7 MB/s eta 0:00:02
   -------- ------------------------------- 8.2/38.8 MB 30.9 MB/s eta 0:00:01
   ----------- ---------------------------- 11.5/38.8 MB 50.1 MB/s eta 0:00:01
   -------------- ------------------------- 14.4/38.8 MB 65.6 MB/s eta 0:00:01
   ------------------ --------------------- 17.7/38.8 MB 72.6 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### eval anomaly

In [19]:
!python "evalAnomaly.py" --input 'C:/Users/vcata\Downloads\dataset_ObstacleTrack/images/*.*' --loadDir "C:/Users/vcata/Desktop/Polito/AML/AnomalySegmentation" --loadWeights "/save/erfnet_training1/model_best.pth" --subset val

Loading model: C:/Users/vcata/Desktop/Polito/AML/AnomalySegmentationerfnet.py
Loading weights: C:/Users/vcata/Desktop/Polito/AML/AnomalySegmentation/save/erfnet_training1/model_best.pth
Model and weights LOADED successfully
Elaborazione immagine: C:\Users\vcata\Downloads\dataset_ObstacleTrack\images\validation_1.webp
Parent:  C:\Users\vcata\Downloads\dataset_ObstacleTrack
Percorso iniziale della maschera: C:\Users\vcata\Downloads\dataset_ObstacleTrack\labels_masks\validation_1_labels_semantic.png
Percorso immagine con ostacolo: C:\Users\vcata\Downloads\dataset_ObstacleTrack\images\validation_1_color.webp
Valori iniziali nella maschera: [  0   1 255]
Valori dopo trasformazione: [  0   1 255]
(1080, 1920)
Append
Elaborazione immagine: C:\Users\vcata\Downloads\dataset_ObstacleTrack\images\validation_10.webp
Parent:  C:\Users\vcata\Downloads\dataset_ObstacleTrack
Percorso iniziale della maschera: C:\Users\vcata\Downloads\dataset_ObstacleTrack\labels_masks\validation_10_labels_semantic.png


C:\Users\vcata\Desktop\Polito\AML\AnomalySegmentation\eval\evalAnomaly.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = load_my_state_dict(model, torch.load(weig

In [ ]:
import os
directory = "C:/Users/vcata/Downloads/dataset_ObstacleTrack/images/"
print(f"Contenuto della directory: {os.listdir(directory)}")

In [26]:
from pathlib import Path
base_path = Path("C:/Users/vcata/Downloads/dataset_ObstacleTrack/images")
files = list(base_path.glob("*.webp"))
print(f"File trovati con pathlib: {[str(file) for file in files]}")

File trovati con pathlib: ['C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_1.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_2.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_3.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_4.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_5.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_carton_6.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_dog_1.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_dog_10.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_dog_2.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_dog_3.webp', 'C:\\Users\\vcata\\Downloads\\dataset_ObstacleTrack\\images\\curvy-street_dog_4.webp